In [ ]:
import pandas as pd

In [ ]:
# Get data Function
# Written as functions for testing in the future and if the source changes the function calls are still valid
def get_enrollment_data() -> pd.DataFrame:
    data = pd.read_excel("enrollments.xlsx")
    return pd.DataFrame(data)

def get_gdp_data() -> pd.DataFrame:
    data = pd.read_csv("Countries_GDP_1960-2020.csv")
    return pd.DataFrame(data)